# Trabalho ENGA75

### Autor: Allan Almeida

In [ ]:
from ur5 import UR5, forward_kinematics
from math import pi
import numpy as np
ur5 = UR5()
PI = pi

## Etapa A1

Implementar a Cinemática Direta e Inversa do Robô UR5. Validar a modelagem com o auxílio do
simulador para um conjunto de valores das variáveis das juntas (Cinemática Direta) e para um conjunto de
posições e orientações da garra (Cinemática Inversa)

#### Cinemática Direta

In [ ]:
def validate_fk(ang):
    fk, _ = forward_kinematics(ang)    # Calcular cinemática direta a partir dos ângulos
    print('Cinemática Direta: ')
    print(fk)
    ur5.move_to_config(ang)     # Mover para a posição
    print('Ground Truth: ')
    gt = ur5.get_ground_truth()     # Obter Ground Truth
    print(gt)
    print('Erro: ', np.linalg.norm(fk-gt)/np.linalg.norm(gt)*100, '%')  # Calcular e printar erro

In [ ]:
validate_fk([0, 0, 0, 0, 0, 0])

In [ ]:
validate_fk([.5*PI, .05*PI, .45*PI, 0, -.5*PI, .5*PI])

In [ ]:
validate_fk([-.5*PI, -.05*PI, -.45*PI, 0, .5*PI, .5*PI])

In [ ]:
ur5.actuate_gripper(close=0)

In [ ]:
ur5.move_to_config([0, 0, 0, 0, 0, 0])

In [ ]:
ur5.move_to_config([.5*pi, .05*pi, .45*pi, 0, -.5*pi, .5*pi])

In [ ]:
ur5.move_to_config([.4*pi, .18*pi, .57*pi, 0.25*pi, 0.8, PI/2])

In [ ]:
# bottle = supervisor.getFromDef('bottle')

In [ ]:
# bottle.getField('translation').setSFVec3f([-1.467559345817077, 1.7836801495521285e-06, 0.9004484925267724])

In [ ]:
# bottle.getField('translation').getSFVec3f()